# Flickr API usage and usefull functions

API reference: https://www.flickr.com/services/api/ <br>
Python API: https://github.com/sybrenstuvel/flickrapi <br>
API allows for 3600 calls per hour.

**Image sizes (longest edge)** / https://www.flickr.com/services/api/misc.urls.html

- o: original
- k: large 2048
- h: large 1600
- b: large 1024
- c: medium 800
- z: medium 640 => COCO format
- : medium 500
- n: small 320
- m: small 240
- t: thumbnail 100
- q: square 150
- s: square 75

In [7]:
from configs import cfg
import os
from pathlib import Path
import glob
import json
from pprint import pprint
from PIL import Image
import flickrapi

KEY = cfg.CRED.FLICKR_KEY
SECRET = cfg.CRED.FLICKR_SECRET
flickr = flickrapi.FlickrAPI(KEY, SECRET)


In [8]:
# photo info json
pprint(json.loads(flickr.photos.getInfo(photo_id='3225245640', format="json")))

{'photo': {'comments': {'_content': '8'},
           'dates': {'lastupdate': '1533162397',
                     'posted': '1232880468',
                     'taken': '2009-01-24 18:01:30',
                     'takengranularity': 0,
                     'takenunknown': 0},
           'dateuploaded': '1232880468',
           'description': {'_content': "<s>I was driving down to my parents' "
                                       'house today, and as I was stopped at a '
                                       'light next to the Franktown Fire '
                                       "Department, I couldn't help but notice "
                                       'a group of firefighters setting a '
                                       'couple of cars next to the firehouse '
                                       'on fire.  Naturally, I parked as '
                                       'quickly as I could and went to take '
                                       'some pictures.</s>\n'

# Useful functions

In [9]:
def authenticate(permission:str):
    """Auhtenticate with given permission.
    
    options: 'read', 'write', 'delete'
    documentation: https://www.flickr.com/services/api/auth.oauth.html
    """
    import webbrowser
    flickr = flickrapi.FlickrAPI(KEY, SECRET)
    
    # Only do this if we don't have a valid token already
    if not flickr.token_valid(perms=permission):

        # Get a request token
        flickr.get_request_token(oauth_callback='oob')

        # Open a browser at the authentication URL. Do this however
        # you want, as long as the user visits that URL.
        authorize_url = flickr.auth_url(perms=permission)
        webbrowser.open_new_tab(authorize_url)

        # Get the verifier code from the user. Do this however you
        # want, as long as the user gives the application the code.
        verifier = str(input('Verifier code: '))

        # Trade the request token for an access token
        flickr.get_access_token(verifier)

In [20]:
# needs write permission, see authenticate function above
authenticate('write')

def add_to_gallery(gallery_id:int, image_ids:list, fav=False):
    """
    Adds images to a flickr gallery. The gallery id is numeric and the last part of the URL. Needs write permission.
    Returns 2 lists: added and failed-to-add image ids.
    If fav = True, photo is also added to favorites.
    """
    # needs loop with pagination if gallery is > 500 pictures
    photos_in_gallery = flickr.galleries.getPhotos(gallery_id=gallery_id, per_page=500, format="json")
    in_gallery = [photo['id'] for photo in json.loads(photos_in_gallery)['photos']['photo']]
    image_ids_gallery = list(set(image_ids) - set(in_gallery))  # drop already added images
    
    success, fail = [], []
    for id in image_ids_gallery:
        # !! limit is 3600 queries per hour
        # some photo owners do not allow to add pictures to galleries; returns "Error: 5: Failed to add photo"
        try:
            flickr.galleries.addPhoto(gallery_id=gallery_id, photo_id=id)
            success.append(id)
        except flickrapi.FlickrError as e:
            print(f'{e}: {id}')
            fail.append(id)
    if fav:
        for id in image_ids:
            try:
                flickr.favorites.add(photo_id=id)
            except flickrapi.FlickrError as e:
                print(f'fav: {e}: {id}')
            
    return success, fail
        
# success, fail = add_to_gallery(72157721019313785, my_ids, fav=False)

In [5]:
def get_url(id:str, size_suffix:str=''):
    """Gets URL for given image id and size.

    args:
    id: flickr image id
    size: defines image size with suffix: https://www.flickr.com/services/api/misc.urls.html
    """
    get_sizes = json.loads(flickr.photos.getSizes(photo_id=id, format="json"))
    found = False
    last_size_url = None
    for size in get_sizes['sizes']['size']:
        last_size_url = size['source']
        _size_suffix = size['source'].split('_')[-1].split('.')[0]  # gets (size) suffix from image source link
        if _size_suffix == size_suffix:
            url = size['source']
            found = True
            break
        elif len(_size_suffix) > 1 and size_suffix == '':  # standard format without suffix in link
            url = size['source']
            found = True
            break
    if not found:   
        url = last_size_url
        suffix = last_size_url.split('_')[-1].split('.')[0]
        print(f'Link/size not found for {id}; biggest image URL available saved, suffix: {suffix if len(suffix)==1 else "none (default)"}')
    return url

def download_image(url:str, output_dir:str, file_name:str):
    """Donwloads image from URL.
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(url, Path(output_dir, file_name))
    
def remove_metadata_img(img_path:str, quality=75):
    """Removes all metadata of an image. Will also compress the image (PIL default is quality=75).
    """
    image = Image.open(img_path)
    data = list(image.getdata())
    img_no_meta = Image.new(image.mode, image.size)
    img_no_meta.putdata(data)
    img_no_meta.save(img_path, quality=quality)  # default quality=75

def ids_from_files(dir):
    """Retrieve flickr IDs from the file name of downloaded images.
    
    flickr file name format: numericalID_imageSecret_imageSize.jpg, e.g. 50235154168_b3201cd930_o.jpg
    """
    return set([x.name.split('_')[0] for x in Path(dir).glob("*.jpg")])


In [12]:
def ids_from_files(dir):
    """Retrieve flickr IDs from the file name of downloaded images.
    
    flickr file name format: numericalID_imageSecret_imageSize.jpg, e.g. 50235154168_b3201cd930_o.jpg
    """
    return set([x.name.split('_')[0] for x in Path(dir).glob("*.jpg")])

my_ids = ids_from_files('/Users/john/datasets/fireground_pose_raw')
# print(len(my_ids))